## Imports

In [232]:
import numpy as np
import pandas as pd
import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Train

In [233]:
train_df = pd.read_csv('../input/train.csv')
train_df['DateTrap'] = train_df['Date']+train_df['Trap']
train_df = train_df[['Date', 'Species', 'Trap', 'Latitude', 'Longitude','NumMosquitos', 'WnvPresent']]
train_df.Date = pd.to_datetime(train_df.Date)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 7 columns):
Date            10506 non-null datetime64[ns]
Species         10506 non-null object
Trap            10506 non-null object
Latitude        10506 non-null float64
Longitude       10506 non-null float64
NumMosquitos    10506 non-null int64
WnvPresent      10506 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 574.6+ KB


In [234]:
train_df['Species'].value_counts()

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

In [235]:
train_df = train_df[(train_df['Species'] == 'CULEX PIPIENS/RESTUANS') | (train_df['Species'] == 'CULEX RESTUANS') | (train_df['Species'] == 'CULEX PIPIENS')]

In [236]:
train_df['Species'].value_counts()

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
Name: Species, dtype: int64

In [237]:
train_df.loc[:,'Pipien'] = train_df['Species'].map(lambda s: 1 if 'PIPIENS' in s else 0)
train_df.loc[:,'Restuans'] = train_df['Species'].map(lambda s: 1 if 'RESTUANS' in s else 0)

In [238]:
train_df.head()

,Date,Species,Trap,Latitude,Longitude,NumMosquitos,WnvPresent,Pipien,Restuans
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,1,0,1,1
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,1,0,0,1
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,1,0,0,1
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,1,0,1,1
4,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,4,0,0,1


In [239]:
train_df.drop('Species', axis=1, inplace = True)

In [240]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10191 entries, 0 to 10505
Data columns (total 8 columns):
Date            10191 non-null datetime64[ns]
Trap            10191 non-null object
Latitude        10191 non-null float64
Longitude       10191 non-null float64
NumMosquitos    10191 non-null int64
WnvPresent      10191 non-null int64
Pipien          10191 non-null int64
Restuans        10191 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 716.6+ KB


In [241]:
train_df['WnvPresent'].value_counts()

0    9640
1     551
Name: WnvPresent, dtype: int64

In [242]:
train_df = train_df[(train_df['Date'].dt.month > 6) | ((train_df['Date'].dt.month == 6) & (train_df['Date'].dt.day > 27))]

In [243]:
train_df['WnvPresent'].value_counts()

0    8273
1     551
Name: WnvPresent, dtype: int64

In [195]:
train_df.to_csv('../input/transformed_input/train_clean_owen', index=False)

### Aggregator

In [244]:
traps = train_df.Trap.values.tolist()

In [263]:
li = []

for trap in traps:
    df = train_df[train_df['Trap']==trap]
    dates = df.Date.values
    for date in dates:
        mini_df = df[df['Date']==date]
        li.append({
            'Date':date,
            'Trap':trap,
            'Latitude':mini_df.iloc[0,2],
            'Longitude':mini_df.iloc[0,3],
            'NumMosquitos':mini_df.iloc[0,4].sum(),
            'WnvPresent':mini_df.iloc[0,5].max(),
            'Pipien':mini_df.iloc[0,6].max(),
            'Restuans':mini_df.iloc[0,7].max(),
        })
    train_df = train_df[train_df['Trap']!=trap]

In [265]:
aggregated_train = pd.DataFrame(li)

In [269]:
aggregated_train = aggregated_train.drop_duplicates()

In [272]:
aggregated_train.WnvPresent.value_counts()

0    3649
1     194
Name: WnvPresent, dtype: int64

In [273]:
aggregated_train.to_csv('../input/transformed_input/aggregated_train_clean_owen', index=False)

## Test

In [196]:
test_df = pd.read_csv('../input/test.csv')

In [197]:
test_df = test_df[['Date', 'Species', 'Trap', 'Latitude', 'Longitude']]
test_df.Date = pd.to_datetime(test_df.Date)
test_df = test_df[(test_df['Species'] == 'CULEX PIPIENS/RESTUANS') | (test_df['Species'] == 'CULEX RESTUANS') | (test_df['Species'] == 'CULEX PIPIENS')]

In [198]:
test_df.loc[:,'pipien'] = test_df['Species'].map(lambda s: 1 if 'PIPIENS' in s else 0)
test_df.loc[:,'restuans'] = test_df['Species'].map(lambda s: 1 if 'RESTUANS' in s else 0)
test_df.drop('Species', axis=1, inplace = True)

In [199]:
test_df = test_df[(test_df['Date'].dt.month > 6) | ((test_df['Date'].dt.month == 6) & (test_df['Date'].dt.day > 27))]

In [200]:
test_df.to_csv('../input/transformed_input/test_clean_owen', index=False)

## Spray

In [201]:
spray_df = pd.read_csv('../input/spray.csv')

In [202]:
spray_df.Date = pd.to_datetime(spray_df.Date)
spray_df.drop('Time', axis=1,inplace=True)

In [203]:
spray_df.to_csv('../input/transformed_input/spray_clean_owen', index=False)

## Weather

In [204]:
weather_df = pd.read_csv('../input/weather.csv')
weather_df.head(10)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
5,2,2007-05-03,67,48,58,M,40,50,7,0,-,-,HZ,M,M,M,0.00,29.46,30.12,12.9,6,13.2
6,1,2007-05-04,66,49,58,4,41,50,7,0,0444,1852,RA,0,M,0.0,T,29.31,30.05,10.4,8,10.8
7,2,2007-05-04,78,51,M,M,42,50,M,M,-,-,,M,M,M,0.00,29.36,30.04,10.1,7,10.4
8,1,2007-05-05,66,53,60,5,38,49,5,0,0443,1853,,0,M,0.0,T,29.40,30.10,11.7,7,12.0
9,2,2007-05-05,66,54,60,M,39,50,5,0,-,-,,M,M,M,T,29.46,30.09,11.2,7,11.5


In [205]:
weather_df.Date = pd.to_datetime(weather_df.Date)

In [206]:
s1 = weather_df[weather_df['Station']==1]

In [207]:
s1['SunLight'] = s1['Sunset'].map(lambda string: int(string[0:2]) + int(string[3:4])/60) - s1['Sunrise'].map(lambda string: int(string[0:2]) + int(string[3:4])/60)

/Users/owencallen/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [208]:
s1 = s1[['Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint','WetBulb', 'Heat', 'Cool', 
         'PrecipTotal', 'SunLight','CodeSum']] #'ResultSpeed', 'ResultDir', 'AvgSpeed',

In [209]:
s1.loc[:,'Tavg'] = pd.to_numeric(s1['Tavg'])
s1.loc[:,'Depart'] = pd.to_numeric(s1['Depart'])

In [210]:
s1[s1['WetBulb']=='M']

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,SunLight,CodeSum
848,2009-06-26,86,69,78,7,60,M,0,13,0.00,14.883333,
2410,2013-08-10,81,64,73,0,57,M,0,8,0.00,14.933333,
2412,2013-08-11,81,60,71,-2,61,M,0,6,0.01,14.066667,RA


In [211]:
weather_df.loc[[849,2411,2413],:]

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
849,2,2009-06-26,86,72,79,M,61,67,0,14,-,-,,M,M,M,0.00,29.20,29.83,6.4,4,8.0
2411,2,2013-08-10,81,68,75,M,55,63,0,10,-,-,,M,M,M,0.00,M,30.07,6.0,6,7.4
2413,2,2013-08-11,84,63,74,M,57,64,0,9,-,-,,M,M,M,T,29.42,30.06,4.0,24,5.4


In [212]:
s1.loc[848,'WetBulb'] = 67
s1.loc[2410,'WetBulb'] = 63
s1.loc[2412,'WetBulb'] = 67
s1.loc[:,'WetBulb'] = pd.to_numeric(s1['WetBulb'])

In [213]:
s1.loc[:,'Heat'] = pd.to_numeric(s1['Heat'])
s1.loc[:,'Cool'] = pd.to_numeric(s1['Cool'])

In [214]:
s1['PrecipTotal'] = s1['PrecipTotal'].map(lambda rain: float(rain) if not ('T' in rain) else 0.01)

In [215]:
s1['Rain'] = s1['CodeSum'].map(lambda s: 1 if 'RA' in s else 0)

In [216]:
s1['Mist'] = s1['CodeSum'].map(lambda s: 1 if 'BR' in s else 0)

In [217]:
s1['Haze'] = s1['CodeSum'].map(lambda s: 1 if 'HZ' in s else 0)

In [218]:
s1['Drizzle'] = s1['CodeSum'].map(lambda s: 1 if 'DZ' in s else 0)

In [219]:
s1['Fog'] = s1['CodeSum'].map(lambda s: 1 if 'FG' in s else 0)

In [220]:
s1.drop('CodeSum',axis=1, inplace=True)

In [221]:
s1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 0 to 2942
Data columns (total 16 columns):
Date           1472 non-null datetime64[ns]
Tmax           1472 non-null int64
Tmin           1472 non-null int64
Tavg           1472 non-null int64
Depart         1472 non-null int64
DewPoint       1472 non-null int64
WetBulb        1472 non-null int64
Heat           1472 non-null int64
Cool           1472 non-null int64
PrecipTotal    1472 non-null float64
SunLight       1472 non-null float64
Rain           1472 non-null int64
Mist           1472 non-null int64
Haze           1472 non-null int64
Drizzle        1472 non-null int64
Fog            1472 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(13)
memory usage: 235.5 KB


In [222]:
s1.to_csv('../input/transformed_input/weather_clean_owen', index=False)